In [1]:
# Install compatible OpenCV version with WeChatQRCode
%pip install opencv-python==4.5.2.52
%pip install opencv-contrib-python==4.5.2.52

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np

In [3]:
def convertScale(img, alpha, beta):
    # Add bias and gain to an image with saturation arithmetics. Unlike
    # cv2.convertScaleAbs, it does not take an absolute value, which would lead to
    # nonsensical results (e.g., a pixel at 44 with alpha = 3 and beta = -210
    # becomes 78 with OpenCV, when in fact it should become 0).

    new_img = img * alpha + beta
    new_img[new_img < 0] = 0
    new_img[new_img > 255] = 255
    return new_img.astype(np.uint8)

# Automatic brightness and contrast optimization with optional histogram clipping
def autoBrightnessAndContrast(image, clip_hist_percent):

    # Calculate grayscale histogram
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    hist_size = len(hist)

    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))

    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0

    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1

    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1

    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha

    auto_result = convertScale(image, alpha=alpha, beta=beta)
    return auto_result

In [4]:
def QRreader(path):

    # declare QR result variable
    qrData = None
    
    # gray scale for faster computation
    gray = cv2.imread(path, 0)

    if gray is not None:

        # stop program if no QR code detected
        try:
            h, w = gray.shape[:2]  
            
            # resize for faster decoding with optional value
            resizeTarget = 240
            ratio = resizeTarget / w
            resize = cv2.resize(gray, (resizeTarget, round(h * ratio)), interpolation = cv2.INTER_AREA)

            # deblur
            gaussian = cv2.GaussianBlur(resize, (11, 11), 10.0)
            unsharp = cv2.addWeighted(resize, 8, gaussian, -7, 0)

            # automatic brightness and contrast optimization with optional histogram clipping
            brightness_contrast = autoBrightnessAndContrast(unsharp, 5)

            # binarization
            _, bin = cv2.threshold(brightness_contrast, 0, 255, cv2.THRESH_OTSU)
            
            # decode using OpenCV WechatQRCode
            detector = cv2.wechat_qrcode_WeChatQRCode("./model/detect.prototxt",
                                                      "./model/detect.caffemodel",
                                                      "./model/sr.prototxt",
                                                      "./model/sr.caffemodel")

            result = detector.detectAndDecode(bin)

            if len(result[0]) != 0:
                qrData = result[0][0]        
        except:
            qrData = None
        
    return qrData, bin

In [5]:
import matplotlib.pyplot as plt

sourceFolder = "./img_qrcode_guiQA"

imgNum = 0
decodeSuccess = 0

# Decode a series of QR code images in "sourceFolder"
for f in os.listdir(sourceFolder):
    try:
        imgPath = os.path.join(sourceFolder, f)
        res, img = QRreader(imgPath)

        # # Create figure and axes
        # _, ax = plt.subplots()
        # # Display the image
        # ax.imshow(img, cmap='gray')
        # plt.title(f)
        # plt.axis("off")  
        # plt.show()
        
        if res is not None:
            # print(f"{res}\n")
            decodeSuccess += 1
        imgNum += 1
    except:
        raise AssertionError


print(f"DECODE RATE: {decodeSuccess}/{imgNum}")

DECODE RATE: 40/100
